In [1]:
import os
import pandas as pd
import glob
from pandasql import sqldf

In [2]:
folder_path = '/Users/saurabhmehta/Documents/Tesla/OneDrive_2_1-27-2024'
file_list = glob.glob(folder_path + "/*.csv") 
main_df = pd.DataFrame(pd.read_csv(file_list[0])) 

In [ ]:
#Load all 32 signals files - temperature and power in one data frame

In [3]:
for i in range(1,len(file_list)): 
    data = pd.read_csv(file_list[i]) 
    df = pd.DataFrame(data) 
    main_df = pd.concat([main_df,df],ignore_index=True) 

In [4]:
main_df.shape

(9097164, 8)

In [5]:
print(sqldf("select distinct Signal_name from main_df"))

                signal_name
0  LM_handleNegCoreTempDegC
1           LM_pcbaTempDegC
2  LM_handlePosCoreTempDegC
3             LM_powerLimit


In [ ]:
# High Temperature Components 105 Farenheit  44.56 Celcius

In [6]:
HighTemp_Components=sqldf("select distinct component from main_df where value > 40.56 and signal_name in ('LM_handleNegCoreTempDegC','LM_pcbaTempDegC','LM_handlePosCoreTempDegC')")

In [7]:
HighTemp_Components

,COMPONENT
0,561876100
1,-1227824566
2,-489322524
3,201209198
4,627685066
...,...
187,1525488905
188,-173839783
189,2085589648
190,1613730181


In [ ]:
# Get High temperature Components with Timestamp concatenated that makes the unique key to join to Power Signals and find Max Power per component.

In [8]:
HighTemp_Components_with_TimeStamp= sqldf("select distinct component||timestamp from main_df where value > 40.56 and signal_name in ('LM_handleNegCoreTempDegC','LM_pcbaTempDegC','LM_handlePosCoreTempDegC')")

In [9]:
HighTemp_Components_with_TimeStamp

,component||timestamp
0,5618761001647460249344
1,5618761001647460260345
2,5618761001647460270349
3,5618761001647460280343
4,5618761001647460290343
...,...
2979173,-20697797091647120472952
2979174,-20697797091647120482948
2979175,-20697797091647120524959
2979176,-20697797091647120534964


In [ ]:
# Component with TimeStamp concatenated makes the unique key to join to Power Signals and find Max Power per component.

In [10]:
MaxPower_for_HighTempComponents=sqldf("select component, max(value) from main_df where signal_name in ('LM_powerLimit') and component||timestamp in (select distinct component||timestamp from main_df where value > 40.56 and signal_name in ('LM_handleNegCoreTempDegC','LM_pcbaTempDegC','LM_handlePosCoreTempDegC')) group by component")

In [11]:
MaxPower_for_HighTempComponents

,COMPONENT,max(value)
0,-2141572541,283.294058
1,-2133907815,175.244091
2,-2126979618,359.410656
3,-2088017096,300.092982
4,-2084872135,358.834772
...,...,...
187,2085446917,293.732776
188,2085589648,192.274059
189,2105022125,286.227751
190,2115833853,359.412658


In [ ]:
# Create a Alerts signal DF to join to temperature and power signal DF.

In [5]:
Alerts_DF=pd.read_csv("/Users/saurabhmehta/Documents/Tesla/Tesla_SEIA_analyst_alerts_data.csv")

In [6]:
Alerts_DF.shape

(832, 7)

In [ ]:
# Create Temperature Signal DF and Power Signal DF 

In [7]:
TemperatureSignal_DF=sqldf("select * from  main_df where value > 40.56 and signal_name in ('LM_handleNegCoreTempDegC','LM_pcbaTempDegC','LM_handlePosCoreTempDegC')")


In [8]:
TemperatureSignal_DF.shape

(4426621, 8)

In [9]:
TemperatureSignal_DF.to_csv("/Users/saurabhmehta/Documents/Tesla/TemperatureSignal_DF.csv", index=False)

In [10]:
PowerSignal_DF=sqldf("select * from main_df where signal_name in ('LM_powerLimit')")


In [11]:
PowerSignal_DF.shape

(3582164, 8)

In [12]:
PowerSignal_DF.to_csv("/Users/saurabhmehta/Documents/Tesla/PowerSignal_DF.csv", index=False)

In [ ]:
# Join PowerSignal and TemperatureSignal with Alerts on component and timestamp(rolled up from milliseconds to minute level) 
# milliseconds rolled up to seconds from milliseconds 
# seconds rolled up to minute by using MOD function with 60 divisor to roll up to nearest minute.
# This rollup / rounding of milliseconds to minute is required so we don't drop any alerts since alerts timestamp doesn't align with Signals timestamp
# This will map alerts to all the signals in that minute for that component. This is better than dropping the alert all together

In [18]:
PowerSignal_and_Alerts_DF= sqldf("select a.*, b.alert_name from PowerSignal_DF a left join Alerts_DF b on (a.component=b.component and (FLOOR(a.timestamp/1000)-MOD((FLOOR(a.timestamp/1000)),60))=(FLOOR(b.timestamp/1000)-MOD((FLOOR(b.timestamp/1000)),60)))")

In [21]:
PowerSignal_and_Alerts_DF.to_csv("/Users/saurabhmehta/Documents/Tesla/PowerSignal_and_Alerts_DF.csv", index=False)

In [19]:
print(sqldf("select * from PowerSignal_and_Alerts_DF where alert_name is not null"))

      Unnamed: 0      timestamp        SITE       ASSET   COMPONENT   
0         261585  1647724503503 -1844894889  1477164192 -1702932383  \
1         261586  1647724508485 -1844894889  1477164192 -1702932383   
2         261587  1647724513492 -1844894889  1477164192 -1702932383   
3         261588  1647724518488 -1844894889  1477164192 -1702932383   
4         261589  1647724523602 -1844894889  1477164192 -1702932383   
...          ...            ...         ...         ...         ...   
1899      281328  1647357576395  -977145491 -2098644635  -226928606   
1900      281330  1647357581402  -977145491 -2098644635  -226928606   
1901      281331  1647357586396  -977145491 -2098644635  -226928606   
1902      281333  1647357592402  -977145491 -2098644635  -226928606   
1903      281336  1647357598400  -977145491 -2098644635  -226928606   

        signal_name       VALUE  event_date                  alert_name  
0     LM_powerLimit  307.883444  2022-03-19  LM_a141_cableTempImbalance  

In [20]:
TemperatureSignal_and_Alerts_DF= sqldf("select a.*, b.alert_name from TemperatureSignal_DF a left join Alerts_DF b on (a.component=b.component and (FLOOR(a.timestamp/1000)-MOD((FLOOR(a.timestamp/1000)),60))=(FLOOR(b.timestamp/1000)-MOD((FLOOR(b.timestamp/1000)),60)))")

In [23]:
TemperatureSignal_and_Alerts_DF.to_csv("/Users/saurabhmehta/Documents/Tesla/TemperatureSignal_and_Alerts_DF.csv", index=False)

In [22]:
print(sqldf("select * from TemperatureSignal_and_Alerts_DF where alert_name is not null"))

      Unnamed: 0      timestamp        SITE       ASSET  COMPONENT   
0          88256  1647393307860  -207607826 -1470107556  252179273  \
1          88256  1647393307860  -207607826 -1470107556  252179273   
2          88258  1647393317818  -207607826 -1470107556  252179273   
3          88258  1647393317818  -207607826 -1470107556  252179273   
4          88261  1647393327827  -207607826 -1470107556  252179273   
...          ...            ...         ...         ...        ...   
2081      109990  1647112270200 -2073653851 -2126066089  -46972795   
2082      109992  1647112281191 -2073653851 -2126066089  -46972795   
2083      109993  1647112292202 -2073653851 -2126066089  -46972795   
2084      109996  1647112302199 -2073653851 -2126066089  -46972795   
2085      109997  1647112312202 -2073653851 -2126066089  -46972795   

                   signal_name      VALUE  event_date   
0     LM_handleNegCoreTempDegC  63.369547  2022-03-16  \
1     LM_handleNegCoreTempDegC  63.369547  20